In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
import string


In [2]:
def clean_rpe():
    
    pre = pd.read_csv('./data/rpe.csv',na_values=['None'])
    df = pre
    print('df shape is ', df.shape)
        # traing 
    df.Training = df.Training.replace('Yes', 1)
    df.Training = df.Training.replace('No', 0)
    
    # dummies for SessionType
    dummies= pd.get_dummies(df.SessionType, prefix='SessionType' )
    for col in list((dummies)):
        df.insert(4, col, dummies[col] )

    # for numeric cols, replace nan with mean for each player
    numeric_cols = [
        'Duration',
        'RPE',
        'SessionLoad',
        'DailyLoad',
        'AcuteLoad',
        'ChronicLoad',
        'AcuteChronicRatio',
        'ObjectiveRating',
        'FocusRating']
    
    for col in numeric_cols:
        for player in set(df.PlayerID):
            mean_ = (df[df.PlayerID == player][col].mean())
            df.update( df[df.PlayerID == player][col].fillna(mean_)) 
#             df[df.PlayerID == player][col] = df[df.PlayerID == player][col].replace(np.nan, mean_)
    # dummies for BestOutOfMyself
    dummies= pd.get_dummies(df.BestOutOfMyself, prefix='BestOutOfMyself' )
    for col in list((dummies)):
        df.insert(21, col, dummies[col] )

    df = df.drop(columns = ['SessionType','BestOutOfMyself'])
    df.head()
    return df

In [3]:
def clean_wellness():
    df = pd.read_csv('./data/wellness.csv')

    df = df[['Date', 'PlayerID', 'Fatigue', 'Soreness', 'Desire', 'Irritability','SleepHours', 'SleepQuality', 'Pain', 'Illness', 'Menstruation']]

    df.loc[df['Pain'] == 'Yes', 'Pain'] = 1
    df.loc[df['Pain'] == 'No', 'Pain'] = 0

    df.loc[df['Illness'] == 'Slightly Off', 'Illness'] = 1
    df.loc[df['Illness'] == 'No', 'Illness'] = 0

    df.loc[df['Menstruation'] == 'Yes', 'Menstruation'] = 1
    df.loc[df['Menstruation'] == 'No', 'Menstruation'] = 0


    nRows, nCols = df.shape
    print(nRows,"rows")
    print(nCols,"columns")

    col = ['Fatigue','Soreness','Desire','Irritability','SleepQuality']
    ID_num = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

    df1=df

    from sklearn import preprocessing
    for num in ID_num:
        df1.loc[df1['PlayerID'] == num,'Fatigue']  = 100*preprocessing.normalize([df1.loc[df1['PlayerID'] == num]['Fatigue']])[0]

    for num in ID_num:
        df1.loc[df1['PlayerID'] == num,'Soreness']  = 100*preprocessing.normalize([df1.loc[df1['PlayerID'] == num]['Soreness']])[0]
    for num in ID_num:
        df1.loc[df1['PlayerID'] == num,'Desire']  = 100*preprocessing.normalize([df1.loc[df1['PlayerID'] == num]['Desire']])[0]

    for num in ID_num:
        df1.loc[df1['PlayerID'] == num,'Irritability']  = 100*preprocessing.normalize([df1.loc[df1['PlayerID'] == num]['Irritability']])[0]
    for num in ID_num:
        df1.loc[df1['PlayerID'] == num,'SleepQuality']  = 100*preprocessing.normalize([df1.loc[df1['PlayerID'] == num]['SleepQuality']])[0]
        
    return df1

In [14]:
def merger_two_df():
    df_rpe = clean_rpe()
    df_wellness = clean_wellness()

    df_merge = df_rpe.merge(df_wellness, left_on=['Date','PlayerID'], right_on=['Date','PlayerID'], suffixes=('_left', '_right'))
#     df_merge.Fatigue.unique()
    
    return df_merge

In [15]:
df_merge = merger_two_df()

df shape is  (8860, 14)
5011 rows
11 columns


In [16]:
df_merge

,Date,PlayerID,Training,SessionType_Strength,SessionType_Speed,SessionType_Skills,SessionType_Mobility/Recovery,SessionType_Game,SessionType_Conditioning,SessionType_Combat,...,BestOutOfMyself_Absolutely,Fatigue,Soreness,Desire,Irritability,SleepHours,SleepQuality,Pain,Illness,Menstruation
0,2018-07-21,13,1,0,0,0,1,0,0,0,...,0,6.041221,5.266401,6.685113,5.324349,8.00,5.439786,0,0,0
1,2018-07-21,13,1,0,0,0,0,1,0,0,...,0,6.041221,5.266401,6.685113,5.324349,8.00,5.439786,0,0,0
2,2018-07-21,13,1,0,0,0,0,1,0,0,...,0,6.041221,5.266401,6.685113,5.324349,8.00,5.439786,0,0,0
3,2018-07-20,11,1,0,0,0,0,1,0,0,...,1,7.817631,7.712299,9.295904,5.857702,8.25,6.474391,0,1,0
4,2018-07-20,11,1,0,0,0,0,1,0,0,...,1,7.817631,7.712299,9.295904,5.857702,8.25,6.474391,0,1,0
5,2018-07-20,11,1,0,0,0,1,0,0,0,...,1,7.817631,7.712299,9.295904,5.857702,8.25,6.474391,0,1,0
6,2018-07-20,13,1,0,0,0,0,1,0,0,...,0,6.041221,7.021868,6.685113,5.324349,8.50,5.439786,0,0,0
7,2018-07-20,13,1,0,0,0,0,1,0,0,...,0,6.041221,7.021868,6.685113,5.324349,8.50,5.439786,0,0,0
8,2018-07-20,13,1,0,0,0,1,0,0,0,...,0,6.041221,7.021868,6.685113,5.324349,8.50,5.439786,0,0,0
9,2018-07-19,1,1,0,0,1,0,0,0,0,...,0,7.846154,6.513760,6.162360,5.548069,9.50,7.472681,0,0,1
